# **1 - Carregamento dos dados tratados**

In [1]:
import pandas as pd

df = pd.read_csv('dados_tratados.csv')
df.head()

,Cliente_Inativo,Sexo,Aposentado,Casado,Dependentes,Meses_Contrato,Assinatura_Telefone,Assinatura_MultiTelefone,Assinatura_Internet,Assinatura_Seguro_online,...,Assinatura_Seguro_dispositivo,Assinatura_Suporte_tecnico,Assinatura_TV,Assinatura_Filmes,Tipo_Contrato,Fatura_Online,Forma_Pagamento,Valor_Mensal,Valor_Diario,Valor_Anual
0,0,0,0,0,0,9,0,0,0,0,...,0,0,0,0,0,0,0,65.6,2.186667,593.30
1,0,1,0,1,1,9,0,1,0,0,...,0,1,1,1,1,1,0,59.9,1.996667,542.40
2,1,1,0,1,1,4,0,0,1,0,...,1,1,1,0,1,0,1,73.9,2.463333,280.85
3,1,1,1,0,1,13,0,0,1,0,...,1,1,0,1,1,0,1,98.0,3.266667,1237.85
4,1,0,1,0,1,3,0,0,1,0,...,0,0,0,0,1,0,0,83.9,2.796667,267.40


# **2 - Análise quantitativa dos cancelamentos**

In [2]:
print('----------------------------------')
print('Quantidade de clientes ativos (No) e inativos (Yes)')
print(df.Cliente_Inativo.value_counts())
print('----------------------------------')
print('Quantidade percentual de clientes ativos (No) e inativos (Yes)')
print(df.Cliente_Inativo.value_counts(normalize=True).map("{:.1%}".format))
print('----------------------------------')

----------------------------------
Quantidade de clientes ativos (No) e inativos (Yes)
0    5163
1    1869
Name: Cliente_Inativo, dtype: int64
----------------------------------
Quantidade percentual de clientes ativos (No) e inativos (Yes)
0    73.4%
1    26.6%
Name: Cliente_Inativo, dtype: object
----------------------------------


# **3 -  Análise correlacional entre os dados**

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## **3.1 - Análise da evasão de clientes**

In [82]:
# Criadno um dataframe temporário para plotar o gráfico de evasão de clientes
df_inativo = df.Cliente_Inativo.value_counts(normalize=True)*100
df_inativo = df_inativo.to_frame().reset_index()
df_inativo.columns = ['Situação do cliente', 'Percentual']
df_inativo['Situação do cliente'] = df_inativo['Situação do cliente'].map({0:'Ativo', 1:'Inativo'})
df_inativo['Percentual'] = df_inativo['Percentual'].round(1)

fig = px.bar(df_inativo, x = df_inativo['Situação do cliente'], y = df_inativo['Percentual'], color = "Situação do cliente", text='Percentual',
    title="Avaliação da taxa de evasão dos clientes")
fig.update_traces(textfont_size=16, textangle=0, textposition="inside", cliponaxis=False, opacity=0.8, marker_line_width=1.5,
    marker_line_color = 'rgb(0,0,0)')
fig.show()

## **3.2 - Perfil de gastos dos clientes**

### **3.2.1 - Perfil de gasto mensal dos clientes**

In [119]:
# Avaliação em relação ao custo mensal do plano
df_gastos = df.Valor_Mensal.value_counts()
df_gastos = df_gastos.to_frame().reset_index()
df_gastos.columns = ['Valor mensal do plano', 'Quantidade']
df_gastos['Situação do cliente'] = df.Cliente_Inativo
df_gastos['Situação do cliente'] = df_gastos['Situação do cliente'].map({0:'Ativo', 1:'Inativo'})


fig = px.histogram(df_gastos, x="Valor mensal do plano", color = "Situação do cliente", nbins = 10,
    title="Situação dos clientes em relação ao custo mensal do plano")
fig.update_traces( opacity=0.8, marker_line_width=1.5, marker_line_color = 'rgb(0,0,0)')
fig.show()

### **3.2.2 - Perfil de gasto total dos clientes**

In [128]:
# # Avaliação em relação ao custo anual total do plano
df_gastos = df.Valor_Anual.value_counts()
df_gastos = df_gastos.to_frame().reset_index()
df_gastos.columns = ['Valor anual total do plano', 'Quantidade']
df_gastos['Situação do cliente'] = df.Cliente_Inativo
df_gastos['Situação do cliente'] = df_gastos['Situação do cliente'].map({0:'Ativo', 1:'Inativo'})


fig = px.histogram(df_gastos, x="Valor anual total do plano", color = "Situação do cliente", nbins = 10,
    title="Situação dos clientes em relação ao custo anual total do plano")
fig.update_traces( opacity=0.8, marker_line_width=1.5, marker_line_color = 'rgb(0,0,0)')
fig.show()

## **3.3 - Tempo de contrato**

In [129]:
df_contrato = df[['Cliente_Inativo', "Meses_Contrato"]]
df_contrato

,Cliente_Inativo,Meses_Contrato
0,0,9
1,0,9
2,1,4
3,1,13
4,1,3
...,...,...
7027,0,13
7028,1,22
7029,0,2
7030,0,67
